In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.optim.lr_scheduler import StepLR
from timm import create_model
import pandas as pd
import copy
import time
from tqdm import tqdm
import os

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

train_dataset = datasets.ImageFolder('C:/Users/liamcee/Documents/farbruh/fer13pluh/train', transform=transform)
test_dataset = datasets.ImageFolder('C:/Users/liamcee/Documents/farbruh/fer13pluh/test', transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

num_classes = len(train_dataset.classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model = create_model('swin_tiny_patch4_window7_224', pretrained=False, num_classes=num_classes)
model.load_state_dict(torch.load('best_swin_fer13.pth'))
model = model.to(device)

C:\Users\liamcee\AppData\Local\Temp\ipykernel_22892\2156865424.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_swin_fer13.pth'))


In [4]:
if os.path.exists("swin_fer13_metrics.csv"):
    df_metrics = pd.read_csv("swin_fer13_metrics.csv")
    start_epoch = len(df_metrics)
    
    train_losses = df_metrics['train_loss'].tolist()
    train_accuracies = df_metrics['train_accuracy'].tolist()
    test_losses = df_metrics['test_loss'].tolist()
    test_accuracies = df_metrics['test_accuracy'].tolist()
    
    best_acc = max(test_accuracies)
else:
    start_epoch = 0
    train_losses, train_accuracies = [], []
    test_losses, test_accuracies = [], []
    best_acc = 0.0

best_model_wts = copy.deepcopy(model.state_dict())

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-5, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=2, gamma=0.1)

In [6]:
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

best_acc = 0.0
best_model_wts = copy.deepcopy(model.state_dict())

def train(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    loop = tqdm(loader, desc="Training", leave=False)

    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        loop.set_postfix(loss=loss.item(), acc=100.0 * correct / total)

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100.0 * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)
    print(f"Train Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

def evaluate(model, loader, criterion, epoch):
    global best_acc, best_model_wts
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    loop = tqdm(loader, desc="Evaluating", leave=False)

    with torch.no_grad():
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            loop.set_postfix(loss=loss.item(), acc=100.0 * correct / total)

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100.0 * correct / total
    test_losses.append(epoch_loss)
    test_accuracies.append(epoch_acc)

    print(f"Test  Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

    # Save best model
    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(best_model_wts, 'best_swin_fer13.pth')
        print(f"best model saved with accuracy: {best_acc:.2f}%")

In [ ]:
epochs = 10
for epoch in range(start_epoch, start_epoch + epochs):
    print(f"\nEpoch {epoch+1}/{start_epoch + epochs}")
    start_time = time.time()

    train(model, train_loader, optimizer, criterion)
    evaluate(model, test_loader, criterion, epoch)

    scheduler.step()
    
    elapsed = time.time() - start_time
    print(f"Epoch Time: {elapsed:.2f} seconds")


Epoch 21/30


Train Loss: 0.5835, Accuracy: 79.16%


Test  Loss: 0.5229, Accuracy: 80.69%
best model saved with accuracy: 80.69%
Epoch Time: 374.41 seconds

Epoch 22/30


Train Loss: 0.4033, Accuracy: 84.81%


Test  Loss: 0.4942, Accuracy: 82.63%
best model saved with accuracy: 82.63%
Epoch Time: 368.43 seconds

Epoch 23/30


...

Test  Loss: 0.8880, Accuracy: 83.29%
best model saved with accuracy: 83.29%
Epoch Time: 369.34 seconds

Epoch 29/30


Train Loss: 0.0476, Accuracy: 98.40%


Test  Loss: 0.8789, Accuracy: 83.17%
Epoch Time: 367.50 seconds

Epoch 30/30


Train Loss: 0.0424, Accuracy: 98.54%


Test  Loss: 0.9406, Accuracy: 83.25%
Epoch Time: 370.40 seconds


In [ ]:
metrics = {
    'epoch': list(range(1, len(train_losses) + 1)),
    'train_loss': train_losses,
    'train_accuracy': train_accuracies,
    'test_loss': test_losses,
    'test_accuracy': test_accuracies
}

df_metrics = pd.DataFrame(metrics)
df_metrics.to_csv("swin_fer13_metrics.csv", index=False)

In [ ]:
torch.save(model, "ferswin_R_full")